## Covid-19 Analytics

In [72]:
# Import Libraries 
import numpy as np
import json 
import pandas as pd
import glob 
import os

In [79]:
# get current working directory
path = os.getcwd() + '..'

In [190]:
# set the base directory.
BASE_DIR = os.path.join( os.path.dirname(path), '' )
# data set folder 
DATASET_DIR  =  BASE_DIR + 'dataset/CORD-19-research-challenge'
# data directories list
DATA_PATH_LIST = [DATASET_DIR +str("/")+ i for i in os.listdir(DATASET_DIR) ]
# get the name of the important folder from the folder names
DATA_FOCUS = [i.split('/')[-1] for i in DATA_PATH_LIST ] 
# deletee the unimportant subset
del DATA_FOCUS[2:5], DATA_FOCUS[3]
DATA_FOCUS 

['noncomm_use_subset', 'comm_use_subset', 'custom_license', 'biorxiv_medrxiv']

In [372]:
folder_contents = {}
for i in DATA_PATH_LIST:
    temp = i.split('/')[-1]
    if temp in  DATA_FOCUS:
        filename = i+'/'+temp+'/pdf_json'
        jsonfiles = glob.glob(os.path.join(filename, "*.json"))
        if temp != 'biorxiv_medrxiv':
            filename2 = i+'/'+temp+'/pmc_json'
            jsonfiles2 = glob.glob(os.path.join(filename2, "*.json"))
            folder_contents[temp+'_pmc']  = jsonfiles
        folder_contents[temp]  = jsonfiles
    

In [373]:
# keys in the dictionary
folder_contents.keys()

dict_keys(['noncomm_use_subset_pmc', 'noncomm_use_subset', 'comm_use_subset_pmc', 'comm_use_subset', 'custom_license_pmc', 'custom_license', 'biorxiv_medrxiv'])

In [374]:
fulldata=[]
#title_doc = {}
for i in  folder_contents.keys():
    print(i)
   
    #if i == 'biorxiv_medrxiv':
    for j in folder_contents[i]:
        paper_id = '';
        temp = '';
        with open(j) as json_data:
            data = json.load(json_data)
            paper_id = data['paper_id']
            temp+= ' ' + data['metadata']['title'] 
            temp += ' :'
            conclusions = ''
            for text in data['body_text']:
                if text['section'] !=  'CONCLUSIONS':
                    temp += 'section: ' + text['section'] + ':'
                    temp += text['text']
                elif text['section'] ==  'CONCLUSIONS':
                    conclusions +=  text['text'] 
                else:
                    temp += text['text']
                
            datadict = {'paper_id': paper_id  , 'text':temp , 'conclusions':conclusions,  'source':i}
            #title_doc[paper_id] = data['metadata']['title']  
        
            fulldata.append(datadict)
            
    

noncomm_use_subset_pmc
noncomm_use_subset
comm_use_subset_pmc
comm_use_subset
custom_license_pmc
custom_license
biorxiv_medrxiv


In [375]:
# make data into data frame
covid = pd.DataFrame(fulldata)
covid.head()

,paper_id,text,conclusions,source
0,1ce2ec3b5d92a3f2ace27a00bde0165ba6761aa0,Interval Between Infections and Viral Hierarc...,,noncomm_use_subset_pmc
1,1e16ae8d036d8775c165e3e64235f90042d59a37,Health response to Hajj mass gathering from e...,,noncomm_use_subset_pmc
2,97a1bf0cca7e6b48364be99814a22a5440c521f3,Next-Generation Sequencing of Small RNAs from...,,noncomm_use_subset_pmc
3,db633ad6a8771828545a8f3d7419722897c4fa9f,:section: INTRODUCTION:Mucosal surfaces form...,,noncomm_use_subset_pmc
4,60522266645f5d89d709c2a6dc0f45a072f64dfd,Elucidation of Bacterial Pneumonia-Causing Pa...,,noncomm_use_subset_pmc


In [391]:
# describe the columns with non empty conclusions
# 1189 articles with conclusions
covid.loc[covid['conclusions'] != '' ]['conclusions'].describe()

count                                                  1189
unique                                                  594
top       Nanotechnological advances should be viewed in...
freq                                                      4
Name: conclusions, dtype: object

In [393]:
covid.to_parquet('cleaned_covid.parquet')

### Meta data load

In [328]:
meta_data = pd.read_csv(DATASET_DIR+'/metadata.csv')

/home/bigdata-lap/anaconda3/envs/cat/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [329]:
meta_data.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,8q5ondtn,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(72)90077-4
1,pzfd0e50,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90355-5
2,22bka3gi,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90356-7
3,zp9k1k3z,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(73)90176-9
4,cjuzul89,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-9343(85)90361-4


In [350]:
# check number of free Nans in sha
len(meta_data) - meta_data['sha'].count()

13015

In [353]:
# change the sha id to paper id
# useful for joining the dataset
meta_data.rename(columns={'sha':'paper_id'}, inplace=True)


In [394]:
# merge covid data with meta data
merged_data = covid.merge(meta_data, on='paper_id', how='inner', suffixes=('_1', '_2'))


In [407]:
# save as feather file
merged_data.to_parquet('merged_data.parquet.gzip',compression='gzip')

In [406]:
# articles with null title
merged_data[merged_data['title'].isnull()]

,paper_id,text,conclusions,source,cord_uid,source_x,title,doi,pmcid,pubmed_id,...,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
22132,15e9f70dc4ea912911e0e77a7f50a604a4f977ef,":section: :172f ancillary tests for, 171-174...",,custom_license_pmc,besya8fi,Elsevier,NaN,10.1016/b978-044306707-5.50067-5,NaN,NaN,...,Unknown,2008-12-31,NaN,Neurology and General Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-044306707-5.50067-5
22133,15e9f70dc4ea912911e0e77a7f50a604a4f977ef,":section: :172f ancillary tests for, 171-174...",,custom_license,besya8fi,Elsevier,NaN,10.1016/b978-044306707-5.50067-5,NaN,NaN,...,Unknown,2008-12-31,NaN,Neurology and General Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-044306707-5.50067-5
23048,5ea24d202f1df5e0c418f920a63fa270c07d46bd,:section: :Aspiration (Continued) radiograph...,,custom_license_pmc,je87ajyi,Elsevier,NaN,10.1016/b978-0-7216-3695-5.50079-1,NaN,NaN,...,Unknown,2006-12-31,NaN,Kendig's Disorders of the Respiratory Tract in...,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-0-7216-3695-5.500...
23049,5ea24d202f1df5e0c418f920a63fa270c07d46bd,:section: :Aspiration (Continued) radiograph...,,custom_license,je87ajyi,Elsevier,NaN,10.1016/b978-0-7216-3695-5.50079-1,NaN,NaN,...,Unknown,2006-12-31,NaN,Kendig's Disorders of the Respiratory Tract in...,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-0-7216-3695-5.500...
24488,c6ef7b36318bd1baf0f3a6638ef571b755d8cfd6,Murine sarcoma virus transformation of BALB/3...,,custom_license_pmc,0dd4yzdh,Elsevier,NaN,10.1016/b978-0-12-253040-1.50023-3,NaN,NaN,...,Unknown,1974-12-31,NaN,The Biology of Animal Viruses,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-0-12-253040-1.500...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59261,36c9a520bf7c7c91972b16849e35000cc7f244f1,:section: :in Haemophilus influenzae infecti...,,custom_license,rr3dqmsm,Elsevier,NaN,10.1016/b978-1-4377-1604-7.00516-9,NaN,NaN,...,Unknown,2012-12-31,NaN,Goldman's Cecil Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-1-4377-1604-7.005...
59386,7c9fa5789635376f2227585b77a903a344bd9225,":section: :Aortic stenosis, 677-685, 679f ca...",,custom_license_pmc,2f3mw2jp,Elsevier,NaN,10.1016/b978-032304841-5.50086-8,NaN,NaN,...,Unknown,2008-12-31,NaN,Critical Care Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-032304841-5.50086-8
59387,7c9fa5789635376f2227585b77a903a344bd9225,":section: :Aortic stenosis, 677-685, 679f ca...",,custom_license,2f3mw2jp,Elsevier,NaN,10.1016/b978-032304841-5.50086-8,NaN,NaN,...,Unknown,2008-12-31,NaN,Critical Care Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-032304841-5.50086-8
60898,7449684ff88a860dd7182cb57067869351d4e49f,:section: Index:Notes Cross-reference terms ...,,custom_license_pmc,8vidj8v6,Elsevier,NaN,10.1016/b978-0-12-394447-4.09001-5,NaN,NaN,...,Unknown,2016-12-31,NaN,Encyclopedia of Cell Biology,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/b978-0-12-394447-4.090...


In [408]:
merged_data['abstract'][7]

"BACKGROUND: Bacterial pneumonia occurring after respiratory viral infection is common. However, the predominant bacterial species causing pneumonia secondary to respiratory viral infections other than influenza remain unknown. The purpose of this study was to know whether the pathogens causing post-viral bacterial pneumonia vary according to the type of respiratory virus. METHODS: Study subjects were 5,298 patients, who underwent multiplex real-time polymerase chain reaction for simultaneous detection of respiratory viruses, among who visited the emergency department or outpatient clinic with respiratory symptoms at Ulsan University Hospital between April 2013 and March 2016. The patients' medical records were retrospectively reviewed. RESULTS: A total of 251 clinically significant bacteria were identified in 233 patients with post-viral bacterial pneumonia. Mycoplasma pneumoniae was the most frequent bacterium in patients aged <16 years, regardless of the preceding virus type (p=0.63